In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
CLIENT_ID = '#################################################' # your Foursquare ID
CLIENT_SECRET = '##############################' # your Foursquare Secret

In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Berlin",
    "radius": 2000,
    "limit": 500}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)
d = data.json()["response"]

In [16]:
d["geocode"]

{'what': '',
 'where': 'berlin',
 'center': {'lat': 52.52437, 'lng': 13.41053},
 'displayString': 'Berlin, Germany',
 'cc': 'DE',
 'geometry': {'bounds': {'ne': {'lat': 52.675075983544005,
    'lng': 13.757081929101975},
   'sw': {'lat': 52.33858497674041, 'lng': 13.088542091271195}}},
 'slug': 'berlin',
 'longId': '72057594040878095'}

In [19]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '51c5bc33498e0d1d9282c2ef',
  'name': 'Cuore di vetro',
  'location': {'address': 'Max-Beer-Str. 33',
   'lat': 52.52657678557984,
   'lng': 13.408722634920323,
   'labeledLatLngs': [{'label': 'display',
     'lat': 52.52657678557984,
     'lng': 13.408722634920323}],
   'postalCode': '10119',
   'cc': 'DE',
   'city': 'Berlin',
   'state': 'Berlin',
   'country': 'Deutschland',
   'formattedAddress': ['Max-Beer-Str. 33', '10119 Berlin', 'Deutschland']},
  'categories': [{'id': '4bf58dd8d48988d1c9941735',
    'name': 'Ice Cream Shop',
    'pluralName': 'Ice Cream Shops',
    'shortName': 'Ice Cream',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-51c5bc33498e0d1d9282c2ef-0'}

In [54]:
pd.set_option('max_colwidth',1000)
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location['formattedAddress']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()


found 95 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,51c5bc33498e0d1d9282c2ef,Cuore di vetro,Ice Cream,"[Max-Beer-Str. 33, 10119 Berlin, Deutschland]",10119,52.526577,13.408723
1,5b55e0a9acb00b0039fb45e3,19grams,Coffee Shop,"[Karl-Liebknecht-Str. 13 (Rochstr.), 10178 Berlin, Deutschland]",10178,52.522697,13.407440
2,54bd1e28498eec46cfc17c24,Father Carpenter,Coffee Shop,"[Münzstr. 21 (1. Hof), 10178 Berlin, Deutschland]",10178,52.524469,13.406614
3,5a6f557389e4906a1487e680,THE REED,Restaurant,"[Karl-Liebknecht-Str. 13, 10178 Berlin, Deutschland]",10178,52.522172,13.408408
4,5433fc35498e6283802aac66,P & T - Paper & Tea,Tea Room,"[Alte SchönhauserAllee 50, 10119 Berlin, Deutschland]",10119,52.527097,13.408234


In [55]:
Berlin_center = d["geocode"]["center"]
Berlin_center

{'lat': 52.52437, 'lng': 13.41053}

In [59]:
from folium import plugins

# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[Berlin_center["lat"], Berlin_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_berlin)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_berlin.add_child(plugins.HeatMap(hm_data))

map_berlin

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
